In [1]:
import copy
import re
import pandas as pd
import numpy as np
from experiments import ExperimentModule
from experiments import (
    save_plots, 
    write_row, 
    write_opcounts,
    save_circuit,
    create_dir,
    get_random_state
)
from sanchez_ansatz import SanchezAnsatz
from qiskit import transpile
from qiskit_algorithms.optimizers import SPSA
import matplotlib.pyplot as plt
from itertools import product
from datetime import datetime

%matplotlib inline

In [2]:
results_dir = "results/runs_shell_2"
run_range = list(range(1, 11))
#qubit_range = [3, 4, 5, 6, 7, 8]
#qubit_range = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
qubit_range = [3, 4, 5, 6, 7, 8, 9, 10]

# EPS values are mapped separately
#eps_values = [0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
eps_values = [0.5, 0.1, 0.05, 0.01]

### Random State type

In [4]:
state_type = "random_state"

In [5]:
#[0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
original_results_dict = {
    "num_qubits": [],
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}

modified_results_dict = copy.deepcopy(original_results_dict)

for (r_dir, st_type, num_qubits, eps) in product([results_dir], [state_type], qubit_range, eps_values):
    if not num_qubits in original_results_dict["num_qubits"]: 
        original_results_dict["num_qubits"] += [num_qubits]
        modified_results_dict["num_qubits"] += [num_qubits]

    original_results_dict[eps] += [[]]
    modified_results_dict[eps] += [[]]

    for run_idx in run_range:
        fid_loss_original = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/csv/original_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_original = fid_loss_original[-1][-1] if len(fid_loss_original.shape) > 1 else fid_loss_original[-1]
        original_results_dict[eps][-1] += [data_original.round(4)]

        fid_loss_modified = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/csv/modified_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_modified = fid_loss_modified[-1][-1] if len(fid_loss_modified.shape) > 1 else fid_loss_modified[-1]
        modified_results_dict[eps][-1]+= [data_modified.round(4)]

In [6]:
default_dict = {
    "num_qubits": qubit_range,
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}
mean_original_results_dict = copy.deepcopy(default_dict)
mean_modified_results_dict = copy.deepcopy(default_dict)

stds_original_results_dict = copy.deepcopy(default_dict)
stds_modified_results_dict = copy.deepcopy(default_dict)

for eps in eps_values:
    mean_original_results_dict[eps] = np.mean(original_results_dict[eps], axis=1)
    stds_original_results_dict[eps] = np.std(original_results_dict[eps], axis=1).round(2)

    mean_modified_results_dict[eps] = np.mean(modified_results_dict[eps], axis=1)
    stds_modified_results_dict[eps] = np.std(modified_results_dict[eps], axis=1).round(2)

Original fidloss

In [7]:
df_mean_original = pd.DataFrame(mean_original_results_dict)
df_mean_original = df_mean_original.set_index("num_qubits")
display(df_mean_original)

df_stds_original = pd.DataFrame(stds_original_results_dict)
df_stds_original = df_stds_original.set_index("num_qubits")
df_stds_original

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.05950,0.05950,0.00112,0.00200
4,0.17780,0.17780,0.12390,0.00092
5,0.19080,0.19080,0.17740,0.15114
6,0.22480,0.22480,0.20020,0.17440
7,0.24860,0.24860,0.24340,0.22460
8,0.23640,0.23640,0.70218,0.22630
9,0.32754,0.25290,0.77733,0.70037
10,0.71669,0.72548,0.55841,0.88387


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.04
6,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.04,0.00
9,0.22,0.00,0.03,0.05
10,0.04,0.03,0.05,0.02


In [8]:
print(df_mean_original.to_latex(float_format="%.4f"))

\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.0595 & 0.0595 & 0.0011 & 0.0020 \\
4 & 0.1778 & 0.1778 & 0.1239 & 0.0009 \\
5 & 0.1908 & 0.1908 & 0.1774 & 0.1511 \\
6 & 0.2248 & 0.2248 & 0.2002 & 0.1744 \\
7 & 0.2486 & 0.2486 & 0.2434 & 0.2246 \\
8 & 0.2364 & 0.2364 & 0.7022 & 0.2263 \\
9 & 0.3275 & 0.2529 & 0.7773 & 0.7004 \\
10 & 0.7167 & 0.7255 & 0.5584 & 0.8839 \\
\bottomrule
\end{tabular}



Modified fidloss

In [9]:
df_mean_modified = pd.DataFrame(mean_modified_results_dict)
df_mean_modified = df_mean_modified.set_index("num_qubits")
display(df_mean_modified)

df_stds_modified = pd.DataFrame(stds_modified_results_dict)
df_stds_modified = df_stds_modified.set_index("num_qubits")
df_stds_modified

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.05950,0.05950,0.01033,0.00145
4,0.17420,0.17420,0.12463,0.00088
5,0.18270,0.18270,0.71420,0.14478
6,0.21140,0.21140,0.56411,0.71241
7,0.24200,0.24200,0.60720,0.21420
8,0.23310,0.23310,0.30317,0.93535
9,0.25090,0.25090,0.92208,0.65437
10,0.88178,0.87917,0.49002,0.99122


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.02,0.00
4,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.02
6,0.00,0.00,0.06,0.00
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.23,0.01
9,0.00,0.00,0.02,0.07
10,0.02,0.02,0.06,0.00


In [10]:
print(df_mean_modified.to_latex(float_format="%.4f"))

\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.0595 & 0.0595 & 0.0103 & 0.0014 \\
4 & 0.1742 & 0.1742 & 0.1246 & 0.0009 \\
5 & 0.1827 & 0.1827 & 0.7142 & 0.1448 \\
6 & 0.2114 & 0.2114 & 0.5641 & 0.7124 \\
7 & 0.2420 & 0.2420 & 0.6072 & 0.2142 \\
8 & 0.2331 & 0.2331 & 0.3032 & 0.9354 \\
9 & 0.2509 & 0.2509 & 0.9221 & 0.6544 \\
10 & 0.8818 & 0.8792 & 0.4900 & 0.9912 \\
\bottomrule
\end{tabular}



In [11]:
df_mean_original < df_mean_modified


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,False,False,True,False
4,False,False,True,False
5,False,False,True,False
6,False,False,True,True
7,False,False,True,False
8,False,False,False,True
9,False,False,True,False
10,True,True,False,True


### Random Sparse States

In [19]:
state_type = "random_sparse_state"
density = 0.5

In [20]:
#[0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
original_results_dict = {
    "num_qubits": [],
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}

modified_results_dict = copy.deepcopy(original_results_dict)

for (r_dir, st_type, num_qubits, eps) in product([results_dir], [state_type], qubit_range, eps_values):
    if not num_qubits in original_results_dict["num_qubits"]: 
        original_results_dict["num_qubits"] += [num_qubits]
        modified_results_dict["num_qubits"] += [num_qubits]

    original_results_dict[eps] += [[]]
    modified_results_dict[eps] += [[]]

    for run_idx in run_range:
        fid_loss_original = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/density_{density}/csv/original_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_original = fid_loss_original[-1][-1] if len(fid_loss_original.shape) > 1 else fid_loss_original[-1]
        original_results_dict[eps][-1] += [data_original.round(4)]

        fid_loss_modified = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/density_{density}/csv/modified_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_modified = fid_loss_modified[-1][-1] if len(fid_loss_modified.shape) > 1 else fid_loss_modified[-1]
        modified_results_dict[eps][-1]+= [data_modified.round(4)]

In [21]:
default_dict = {
    "num_qubits": qubit_range,
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}
mean_original_results_dict = copy.deepcopy(default_dict)
mean_modified_results_dict = copy.deepcopy(default_dict)

stds_original_results_dict = copy.deepcopy(default_dict)
stds_modified_results_dict = copy.deepcopy(default_dict)

for eps in eps_values:
    mean_original_results_dict[eps] = np.mean(original_results_dict[eps], axis=1)
    stds_original_results_dict[eps] = np.std(original_results_dict[eps], axis=1).round(2)

    mean_modified_results_dict[eps] = np.mean(modified_results_dict[eps], axis=1)
    stds_modified_results_dict[eps] = np.std(modified_results_dict[eps], axis=1).round(2)

In [22]:
df_mean_original = pd.DataFrame(mean_original_results_dict)
df_mean_original = df_mean_original.set_index("num_qubits")
display(df_mean_original)
display(df_mean_modified)

df_stds_original = pd.DataFrame(stds_original_results_dict)
df_stds_original = df_stds_original.set_index("num_qubits")
#df_stds_original

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.39439,0.37638,0.05481,0.00000
4,0.53770,0.53770,0.30590,0.00005
5,0.52695,0.52588,0.48153,0.38357
6,0.55190,0.55190,0.52440,0.43341
7,0.60200,0.60200,0.62795,0.55412
8,0.60480,0.60480,0.59732,0.57740
9,0.60686,0.64335,0.94220,0.84465
10,0.79685,0.79150,0.66350,0.96626


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.05950,0.05950,0.01033,0.00145
4,0.17420,0.17420,0.12463,0.00088
5,0.18270,0.18270,0.71420,0.14478
6,0.21140,0.21140,0.56411,0.71241
7,0.24200,0.24200,0.60720,0.21420
8,0.23310,0.23310,0.30317,0.93535
9,0.25090,0.25090,0.92208,0.65437
10,0.88178,0.87917,0.49002,0.99122


In [23]:
display(df_stds_original)
display(df_stds_modified)

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.05,0.00,0.16,0.00
4,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.04
6,0.00,0.00,0.00,0.00
7,0.00,0.00,0.05,0.00
8,0.00,0.00,0.00,0.00
9,0.00,0.11,0.02,0.02
10,0.02,0.04,0.04,0.01


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.02,0.00
4,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.02
6,0.00,0.00,0.06,0.00
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.23,0.01
9,0.00,0.00,0.02,0.07
10,0.02,0.02,0.06,0.00


In [24]:
print("--"*100)
print("Original Mean")
print("--"*100)
print(df_mean_original.to_latex(float_format="%.4f"))

print("--"*100)
print("Modified Mean")
print("--"*100)
print(df_mean_modified.to_latex(float_format="%.4f"))

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Original Mean
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.3944 & 0.3764 & 0.0548 & 0.0000 \\
4 & 0.5377 & 0.5377 & 0.3059 & 0.0001 \\
5 & 0.5270 & 0.5259 & 0.4815 & 0.3836 \\
6 & 0.5519 & 0.5519 & 0.5244 & 0.4334 \\
7 & 0.6020 & 0.6020 & 0.6279 & 0.5541 \\
8 & 0.6048 & 0.6048 & 0.5973 & 0.5774 \\
9 & 0.6069 & 0.6433 & 0.9422 & 0.8447 \\
10 & 0.7969 & 0.7915 & 0.6635 & 0.9663 \\
\bottomrule
\end{tabular}

-----------------------------------------------------------------------------------------------------------------

In [25]:
df_mean_original < df_mean_modified

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,False,False,False,True
4,False,False,False,True
5,False,False,True,False
6,False,False,True,True
7,False,False,False,False
8,False,False,False,True
9,False,False,False,False
10,True,True,False,True
